In [1]:
import Control.Monad.Trans.State
import Control.Monad.Trans.Cont
import Control.Monad.Trans.Identity

data Iterator i o r = Result r | Susp o (i -> Iterator i o r)

instance (Show o, Show r) => Show (Iterator i o r) where
  show (Result r) = "Result "++show r
  show (Susp i k) = "Susp "++show i++" <<...>>"

run :: Cont (Iterator i o r) r -> Cont r' (Iterator i o r)
run e = reset $ do { r <- e; return (Result r) }

yield :: o -> Cont (Iterator i o r) i
yield e = shift $ \k -> return (Susp e k)

In [2]:
data Tree a = Null | Node a (Tree a) (Tree a)  deriving (Eq,Show)

walkIO :: Show a => Tree a -> IO ()
walkIO Null = return ()
walkIO (Node c t1 t2) =
  do walkIO t1
     print c
     walkIO t2

walk :: Tree o -> Cont (Iterator i o r) (Tree i)
walk Null = return Null
walk (Node o t1 t2) =
  do t1' <- walk t1
     i   <- yield o
     t2' <- walk t2
     return $ Node i t1' t2'

In [3]:
loop1 :: Show o => Iterator () o r -> IO ()
loop1 (Result _) = return ()
loop1 (Susp o k) = do print o
                      loop1 (k ())

loop2 :: Monad m => Iterator o o r -> m r
loop2 (Result r) = return r
loop2 (Susp o k) = loop2 (k o)

loop3 :: Monad m => Iterator Int o r -> StateT Int m r
loop3 (Result r) = return r
loop3 (Susp _ k) = do modify (+1)
                      i <- get
                      loop3 (k i)

loop4 :: Monad m => Iterator (Int,o) o r -> StateT Int m r
loop4 (Result r) = return r
loop4 (Susp o k) = do modify (+1)
                      i <- get
                      loop4 (k (i,o))

In [4]:
tree1 :: Tree Char
tree1 = Node 'b' (Node 'a' Null Null)
                 (Node 'c' Null Null)

do0 :: IO ()
do0 = walkIO tree1

do1 :: IO ()
do1 = (`runCont` loop1) . run $ walk tree1

do2 :: Monad m => m (Tree Char)
do2 = (`runCont` loop2) . run $ walk tree1

do3 :: Monad m => StateT Int m (Tree Int)
do3 = (`runCont` loop3) . run $ walk tree1

do4 :: Monad m => StateT Int m (Tree (Int,Char))
do4 = (`runCont` loop4) . run $ walk tree1

In [5]:
do0

'a'
'b'
'c'

In [6]:
do1 -- same as do0

'a'
'b'
'c'

In [7]:
do2

Node 'b' (Node 'a' Null Null) (Node 'c' Null Null)

In [8]:
evalState do3 0

Node 2 (Node 1 Null Null) (Node 3 Null Null)

In [9]:
evalState do4 0

Node (2,'b') (Node (1,'a') Null Null) (Node (3,'c') Null Null)